# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import glob
import numpy as np
import os
import pandas as pd
import psycopg2
from typing import List, Tuple
import itertools
from itertools import groupby
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=Ckmerrypos44!")
cur = conn.cursor()


In [3]:
# Print PostgreSQL Connection properties
print (conn.get_dsn_parameters(),"\n")

# Print PostgreSQL version
cur.execute("SELECT version();")
record = cur.fetchone()
print("You are connected to - ", record,"\n")

{'user': 'postgres', 'dbname': 'sparkifydb', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 12.2, compiled by Visual C++ build 1914, 64-bit',) 



In [4]:
# Check tables are in database
# https://stackoverflow.com/questions/10598002/how-do-i-get-tables-in-postgres-using-psycopg2
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('artists',)
('songs',)
('time',)
('songplays',)
('users',)


In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        #print(root, dirs, files)
        #print(root)
        #print(os.path.join(root,'*.json'))
        #print(glob.glob(os.path.join(root,'*.json')))
        files = glob.glob(os.path.join(root,'*.json'))
        #print(files)
        for f in files :
            #print(f)
            all_files.append(os.path.abspath(f))
    
    return all_files

# More on path and glob here
# https://docs.python.org/3/library/os.html
# https://docs.python.org/3/library/glob.html
# glob grabs the files at root, i.e. pathanmes that match pathname

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data')

In [7]:
song_files[1]

'C:\\Python\\projects\\sparkify\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json'

In [8]:
filepath = song_files[0]

In [9]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
#song_data = 

song_columns = ['song_id', 'title', 'artist_id', 'year', 'duration']

song_data = (pd.read_json(filepath, lines=True)
               .loc[:, song_columns]
               .to_numpy()
               #.ravel()
               .tolist())
song_data

[['SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179]]

In [11]:
# Test with just one for deterministic testing
song_columns = ['song_id', 'title', 'artist_id', 'year', 'duration']

song_data = (pd.read_json(filepath, lines=True)
               .loc[:, song_columns]
               .to_numpy()
               .ravel() # Needs ravel() for single insert test
               .tolist())
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
# NEED OT CREATE ARTIST TABLE FIRST
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = 
artist_data

In [12]:
artist_columns = ['artist_id', 
                  'artist_name', 
                  'artist_location', 
                  'artist_latitude', 
                  'artist_longitude']

artist_data =(pd.read_json(filepath, lines=True)
                 #.applymap(lambda x: 'NULL' if np.isnan(x) else x)
                 #.fillna('NULL')
                 .loc[:, artist_columns]
                 .to_numpy()
                 .ravel()
                 .tolist())

artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files('data/log_data')

In [16]:
filepath = log_files[0]

In [17]:
df = pd.read_json(filepath, 
                  convert_dates=['registration', 'ts'],
                  date_unit='ms',  # 13 digits is milliseconds
                  lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,2018-10-30 17:06:06.796,38,None,200,2018-11-01 20:57:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,2018-10-24 01:33:14.796,139,None,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,2018-10-24 01:33:14.796,139,None,200,2018-11-01 21:02:12.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = 
df.head()

In [ ]:
t = 
t.head()

In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

In [18]:
time_columns = ['start_time',
                'hour',
                'day',
                'week',
                'month',
                'year',
                'weekday']

time_data = (pd.read_json(filepath, 
                         convert_dates=['registration', 'ts'],
                         date_unit='ms',
                         lines=True)
               .query("page == 'NextSong'") 
               .assign(start_time=lambda x: x.ts)
               .assign(hour=lambda x: x.ts.dt.hour)
               .assign(day=lambda x: x.ts.dt.day)
               .assign(week=lambda x: x.ts.dt.weekofyear)
               .assign(month=lambda x: x.ts.dt.month)
               .assign(year=lambda x: x.ts.dt.year)
               .assign(weekday=lambda x: x.ts.dt.weekday)
               #.applymap(lambda t: t.strftime('%Y-%m-%d %H:%M:%f') if isinstance(t, pd.Timestamp) else t)
               .loc[:, time_columns]
               .to_numpy()
               .tolist())

In [19]:
time_data

[[Timestamp('2018-11-01 21:01:46.796000'), 21, 1, 44, 11, 2018, 3],
 [Timestamp('2018-11-01 21:05:52.796000'), 21, 1, 44, 11, 2018, 3],
 [Timestamp('2018-11-01 21:08:16.796000'), 21, 1, 44, 11, 2018, 3],
 [Timestamp('2018-11-01 21:11:13.796000'), 21, 1, 44, 11, 2018, 3],
 [Timestamp('2018-11-01 21:17:33.796000'), 21, 1, 44, 11, 2018, 3],
 [Timestamp('2018-11-01 21:24:53.796000'), 21, 1, 44, 11, 2018, 3],
 [Timestamp('2018-11-01 21:28:54.796000'), 21, 1, 44, 11, 2018, 3],
 [Timestamp('2018-11-01 21:42:00.796000'), 21, 1, 44, 11, 2018, 3],
 [Timestamp('2018-11-01 21:52:05.796000'), 21, 1, 44, 11, 2018, 3],
 [Timestamp('2018-11-01 21:55:25.796000'), 21, 1, 44, 11, 2018, 3],
 [Timestamp('2018-11-01 22:23:14.796000'), 22, 1, 44, 11, 2018, 3]]

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [20]:
# MY WAY
for entry in time_data:
    cur.execute(time_table_insert, entry)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

In [21]:
user_columns = ['userId',
                'firstName',
                'lastName',
                'gender',
                'level']

user_data = (pd.read_json(filepath, 
                          convert_dates=['registration', 'ts'],
                          date_unit='ms',
                          lines=True)
                .loc[:, user_columns]
                .to_numpy()
                .tolist())

user_data[0:5]

[[39, 'Walter', 'Frye', 'M', 'free'],
 [8, 'Kaylee', 'Summers', 'F', 'free'],
 [8, 'Kaylee', 'Summers', 'F', 'free'],
 [8, 'Kaylee', 'Summers', 'F', 'free'],
 [8, 'Kaylee', 'Summers', 'F', 'free']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
# Dedupe here for single test (handled in pipeline otehrwise
import itertools

user_data_deduped = list(user_data for user_data, _ in itertools.groupby(user_data))

user_data_deduped

[[39, 'Walter', 'Frye', 'M', 'free'],
 [8, 'Kaylee', 'Summers', 'F', 'free'],
 [10, 'Sylvie', 'Cruz', 'F', 'free'],
 [26, 'Ryan', 'Smith', 'M', 'free'],
 [101, 'Jayden', 'Fox', 'M', 'free']]

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [23]:
# MY WAY
for entry in user_data_deduped:
    cur.execute(user_table_insert, entry)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [31]:
songplays_columns = ['start_time',
                     'userId',
                     'level',
                     'sessionId',
                     'location',
                     'userAgent',
                     'song']

songplays_data = (pd.read_json(filepath, 
                         convert_dates=['registration', 'ts'],
                         date_unit='ms',
                         lines=True)
                    .query("song.to_numpy() != None")
                    .assign(start_time=lambda x: x.ts)
                    #.applymap(lambda t: t.strftime('%Y-%m-%d %H:%M:%f') if isinstance(t, pd.Timestamp) else t)
                    .loc[:, songplays_columns]
                    .to_numpy()
                    .tolist())

In [32]:
songplays_data[0:2]

[[Timestamp('2018-11-01 21:01:46.796000'),
  8,
  'free',
  139,
  'Phoenix-Mesa-Scottsdale, AZ',
  '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"',
  'You Gotta Be'],
 [Timestamp('2018-11-01 21:05:52.796000'),
  8,
  'free',
  139,
  'Phoenix-Mesa-Scottsdale, AZ',
  '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"',
  'Flat 55']]

In [26]:
song_artist_columns = ['artist_id',
                       'song_id',
                       'title']

song_artist_data = (pd.read_json(song_files[0], 
                            lines=True)
                      .loc[:, song_artist_columns]
                      .to_numpy()
                      .tolist())

song_artist_data

[['ARD7TVE1187B99BFB1', 'SOMZWCG12A8C13C480', "I Didn't Mean To"]]

In [27]:
# Make dictionary
song_artist_keys = [item[2] for item in song_artist_data]
song_artist_values = [item[0:2] for item in song_artist_data]

song_artist_dispatch = dict(zip(song_artist_keys, song_artist_values))

song_artist_dispatch["I Didn't Mean To"]

['ARD7TVE1187B99BFB1', 'SOMZWCG12A8C13C480']

In [33]:
# Flatten and dedupe songplays data
# NOT NEEDED FOR THIS CASE
def flatten_log_data(data):
    flat_data = [item for logfile in data for item in logfile]
    dedup_data = list(flat_data for flat_data, _ in itertools.groupby(flat_data))
    return dedup_data

songplays_data_flat = flatten_log_data(songplays_data)

#songplays_data_flat

In [28]:
def process_songplays_data(data_for_songplays, songs_dict):
    processed_data = []
    for index, entry in enumerate(data_for_songplays):
        if entry[6] in songs_dict.keys():
            entry.extend(songs_dict[entry[6]])
            entry.remove(entry[6])
            processed_data.append(entry)
        else:
            entry.extend([None, None])
            entry.remove(entry[6])
            processed_data.append(entry)

    return processed_data

In [33]:
songplays_data_for_insert = process_songplays_data(songplays_data, song_artist_dispatch)

songplays_data_for_insert[0:2]

[[Timestamp('2018-11-01 21:01:46.796000'),
  8,
  'free',
  139,
  'Phoenix-Mesa-Scottsdale, AZ',
  '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"',
  None,
  None],
 [Timestamp('2018-11-01 21:05:52.796000'),
  8,
  'free',
  139,
  'Phoenix-Mesa-Scottsdale, AZ',
  '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"',
  None,
  None]]

In [36]:
# MY WAY
for entry in songplays_data_for_insert:
    cur.execute(songplay_table_insert, entry)
    conn.commit()

---
IGNORE BELOW

In [50]:
dfcopy = df.loc[df['page'] == 'NextSong']
dfcopy.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796,139,Eriatarka,200,2018-11-01 21:11:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796,139,Becoming Insane,200,2018-11-01 21:17:33.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [39]:
# Add a row of data we have to check functionality of song_select
columns = dfcopy.columns
columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [49]:
row_to_add = pd.DataFrame([['Casual', 'Logged In', 'Kaylee', 'F', 5, 'Summers', 218.93179,
              'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', None, None, 123, "I Didn't Mean To",
              200, None, None, 8]], columns=columns)

row_to_add

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Casual,Logged In,Kaylee,F,5,Summers,218.93179,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,None,None,123,I Didn't Mean To,200,None,None,8


In [51]:
dfcopy = dfcopy.append(row_to_add)


In [59]:
dfcopy

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796000,139,You Gotta Be,200,2018-11-01 21:01:46.796000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796000,139,Flat 55,200,2018-11-01 21:05:52.796000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796000,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796000,139,Eriatarka,200,2018-11-01 21:11:13.796000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796000,139,Becoming Insane,200,2018-11-01 21:17:33.796000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796000,139,Congratulations,200,2018-11-01 21:24:53.796000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,2018-10-24 01:33:14.796000,139,Once again,200,2018-11-01 21:28:54.796000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
10,Black Eyed Peas,Logged In,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,2018-10-23 03:43:05.796000,9,Pump It,200,2018-11-01 21:42:00.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
12,Fall Out Boy,Logged In,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,2018-10-31 20:11:47.796000,169,Nobody Puts Baby In The Corner,200,2018-11-01 21:52:05.796000,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
13,M.I.A.,Logged In,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,2018-10-31 20:11:47.796000,169,Mango Pickle Down River (With The Wilcannia Mob),200,2018-11-01 21:55:25.796000,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [62]:
for index, row in dfcopy.iterrows():
    print(row.song, row.artist, round(row.length, 3))

You Gotta Be Des'ree 246.308
Flat 55 Mr Oizo 144.039
Quem Quiser Encontrar O Amor Tamba Trio 177.188
Eriatarka The Mars Volta 380.421
Becoming Insane Infected Mushroom 440.267
Congratulations Blue October / Imogen Heap 241.397
Once again Girl Talk 160.156
Pump It Black Eyed Peas 214.935
Nobody Puts Baby In The Corner Fall Out Boy 200.724
Mango Pickle Down River (With The Wilcannia Mob) M.I.A. 233.717
Eye Of The Tiger Survivor 245.368
I Didn't Mean To Casual 218.932


In [67]:
# Test song_select
for index, row in dfcopy.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.artist, row.song, round(row.length, 3)))
    results = cur.fetchone()
    print(results)
    
# OK

None
None
None
None
None
None
None
None
None
None
None
('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')


In [65]:
# song_select works fine
cur.execute(song_select, ('Casual',"I Didn't Mean To", 218.932))
cur.fetchone()


# TODO implement songplay_data
# Then run all with execute valeus in bulk

('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')

In [20]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    #cur.execute(song_select, (row.song, row.artist, row.length))
    cur.execute(song_select, (row.artist, row.song, round(row.length, 3)))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

ProgrammingError: can't execute an empty query

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [68]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.